In [1]:
import Pkg; 

if split(pwd(),"/")[end] == "notebooks"
    cd(joinpath(@__DIR__, "../"))
    Pkg.activate("Project.toml")
end

using MorphoMol
using StaticArrays
using Rotations
using Distances
using JLD2

  Activating project at `~/Doktor/Code/MorphoMolMonteCarlo`


In [2]:
function get_flat_realization(x, template_mol)
    n_mol = length(x) ÷ 6
    [(hvcat((n_mol), [exp(Rotations.RotationVecGenerator(x[i:i+2]...)) * template_mol .+ x[i+3:i+5] for i in 1:6:length(x)]...)...)...]
end

get_flat_realization (generic function with 1 method)

In [3]:
function get_matched_distances_between_transformation_offsets(template_centers_a, template_centers_b, state_a, state_b)
    function transform_dist(template_centers, state, idx)
        i = 1
        j = 2
        R1 = exp(RotationVecGenerator(state[(i-1) * 6 + 1:(i-1) * 6 + 3]...))
        T1 = state[(i-1) * 6 + 4:(i-1) * 6 + 6]
        R2 = exp(RotationVecGenerator(state[(j-1) * 6 + 1:(j-1) * 6 + 3]...))
        T2 = state[(j-1) * 6 + 4:(j-1) * 6 + 6]
        euclidean(T1 + R1 * template_centers[:,idx], T2 + R2 * template_centers[:,idx])
    end

    [abs(transform_dist(template_centers_a, state_a, i) - transform_dist(template_centers_b, state_b, i)) for i in 1:size(template_centers_a)[2]]
end

function average_offset_distance(template_centers_a, template_centers_b, state_a, state_b)
    n = size(template_centers_a)[2]
    sum(get_matched_distances_between_transformation_offsets(template_centers_a, template_centers_b, state_a, state_b)) / n
end

average_offset_distance (generic function with 1 method)

In [5]:
folder = "assets/hpc_out/"


"assets/hpc_out/"

In [12]:
for file in readdir(folder)
    mt = split(file, "_")[2]
    num = split(file, "_")[3]
    mkpath(joinpath(folder, "$(mt)_$(num)"))
    mv(joinpath(folder, file), joinpath(folder, "$(mt)_$(num)", file))
end

In [13]:
min_path_by_mol = Dict("6r7m" => "", "6sae" => "", "6sag" => "", "2tmv" => "", "1ei7" => "")
E_min_by_mol = Dict("6r7m" => Inf, "6sae" => Inf, "6sag" => Inf, "2tmv" => Inf, "1ei7" => Inf)
mol_types = ["6r7m", "6sae", "6sag", "2tmv", "1ei7"]

rwm_hot_start_folder = "assets/input/jld2/2_low/"
for mol_type in mol_types
    for file in readdir(rwm_hot_start_folder)
        if  split(file, ".")[end] == "jld2" && split(file, "_")[1] == mol_type
            try
                @load "$rwm_hot_start_folder$file" template_mol template_radii x_init
                template_mol = reshape(template_mol,(3,length(template_radii)))
                realization = get_flat_realization(x_init, template_mol)
                E = MorphoMol.Energies.solvation_free_energy(
                    realization, 
                    length(template_radii),
                    [template_radii; template_radii],
                    1.4,
                    MorphoMol.Energies.get_prefactors(1.4, 0.3665),
                    0.0,
                    0.85
                    )
                if E_min_by_mol[mol_type] > E
                    E_min_by_mol[mol_type] = E
                    min_path_by_mol[mol_type] = "$rwm_hot_start_folder/$file"
                end
            catch e
                println(e)
            end
        end
    end
end

rwm_hot_start_folder = "assets/input/jld2/2_low/"
for mol_type in mol_types
    if min_path_by_mol[mol_type] != ""
        @load min_path_by_mol[mol_type] template_mol template_radii x_init
        template_mol = reshape(template_mol,(3,length(template_radii)))
        realization = get_flat_realization(x_init, template_mol)
        E = MorphoMol.Energies.solvation_free_energy(
            realization, 
            length(template_radii),
            [template_radii; template_radii],
            1.4,
            MorphoMol.Energies.get_prefactors(1.4, 0.3665),
            0.0,
            0.85
            )
        template_mol_sim = deepcopy(template_mol)
        x_sim = deepcopy(x_init)
        @load "assets/input/jld2/experimental/2_$(mol_type)_protor.jld2" template_mol template_radii x_init
        println(mol_type, " ", minimum(E), " ", average_offset_distance(template_mol_sim, template_mol, x_sim, x_init))
    end
end


6r7m 8138.924225194982 0.9982893464282008
6sae 8107.279622532144 1.2821182928454404
6sag 8097.730817128498 1.1146356769067605
2tmv 8056.550361360152 21.083570792125695
1ei7 8185.798536861604 24.19202036075877


In [14]:
E_min_by_mol = Dict("6r7m" => Inf, "6sae" => Inf, "6sag" => Inf, "2tmv" => Inf, "1ei7" => Inf)
min_path_by_mol = Dict("6r7m" => "", "6sae" => "", "6sag" => "", "2tmv" => "", "1ei7" => "")
for mol_type in mol_types
    E_min = E_min_by_mol[mol_type]
    min_file_path = ""
    for sf in readdir(folder)
        for file in readdir("$folder$sf")
            if  split(file, ".")[end] == "jld2" && split(file, "_")[2] == mol_type
                try
                    @load "$folder$sf/$file" in_out_data
                    if E_min > minimum(in_out_data.output.Es)
                        E_min = minimum(in_out_data.output.Es)
                        min_file_path = "$folder$sf/$file"
                    end
                catch e
                    println(e)
                end
            end
        end
    end
    min_path_by_mol[mol_type] = min_file_path
end

In [15]:
println(min_path_by_mol)

Dict("6sag" => "assets/hpc_out/6sag_5/3_6sag_5_protor_oj_0_0_os_0_85.jld2", "1ei7" => "", "6r7m" => "", "6sae" => "assets/hpc_out/6sae_13/3_6sae_13_protor_oj_0_0_os_0_85.jld2", "2tmv" => "")


In [16]:
for mol_type in ["6r7m", "6sae", "6sag", "2tmv", "1ei7"]
    if min_path_by_mol[mol_type] != ""
        @load min_path_by_mol[mol_type] in_out_data
        template_mol_sim = in_out_data.input.template_mol
        x_sim = in_out_data.output.states[1]
        
        @load "assets/input/jld2/experimental/2_$(mol_type)_protor.jld2" template_mol template_radii x_init
        println(mol_type, " ", minimum(in_out_data.output.Es), " ", average_offset_distance(template_mol_sim, template_mol, x_sim, x_init))
    end
end

6sae 8106.102486349152 1.1429573909935333
6sag 8092.188608572787 1.1146356769067605
